In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import numpy as np
!pip install -U keras
!pip install tensorflow

  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [ ]:
import math
import random
import keras
from keras.layers import *
from keras.models import Sequential
from keras import Model
from keras import backend as K  
from keras.regularizers import l2
from keras.callbacks import EarlyStopping,ModelCheckpoint
!pip install tensorflow

import tensorflow as tf
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics

In [ ]:
def get_conv_block(input_layer,nFilters,size):
    conv1 = Conv2D(nFilters, size, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(input_layer)
    bn1 = BatchNormalization()(conv1)
    conv2 = Conv2D(nFilters, size, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(bn1)
    bn2 = BatchNormalization()(conv2)
    return bn2
    
def get_UNET(input_layer,nFilters,flag): 

    block1 = get_conv_block(input_layer[0],nFilters,3)
    mp1 = MaxPooling2D(pool_size=(2, 2))(block1)
    dr1 = Dropout(0.1)(mp1)
   
    if(flag==1):
      dr1 = concatenate([dr1,input_layer[1]])

    block2 = get_conv_block(dr1,nFilters*2,3)
    mp2 = MaxPooling2D(pool_size=(2, 2))(block2)
    dr2 = Dropout(0.1)(mp2)

    if(flag==1):
      dr2 = concatenate([dr2,input_layer[2]])
   
    block3 = get_conv_block(dr2,nFilters*4,3)
    mp3 = MaxPooling2D(pool_size=(2, 2))(block3)
    dr3 = Dropout(0.1)(mp3)

    if(flag==1):
      dr3 = concatenate([dr3,input_layer[3]])
       
    block4 = get_conv_block(dr3,nFilters*8,3)
    mp4 = MaxPooling2D(pool_size=(2, 2))(block4)
    dr4 = Dropout(0.1)(mp4)

    conv5 = Conv2D(nFilters*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(dr4)
    conv5 = Conv2D(nFilters*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(conv5)

    up1 = Conv2DTranspose(nFilters*8,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(conv5)
    cat1 = concatenate([block4, up1])
    dr1 = Dropout(0.1)(cat1)
    block5 = get_conv_block(dr1,nFilters*8,3)

    up2 = Conv2DTranspose(nFilters*4,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(block5)
    cat2 = concatenate([block3, up2,])
    dr2 = Dropout(0.1)(cat2)
    block6 = get_conv_block(dr2,nFilters*4,3)
    
    up3 = Conv2DTranspose(nFilters*2,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(block6)
    cat3 = concatenate([block2, up3])
    dr3 = Dropout(0.1)(cat3)
    block7 = get_conv_block(dr3,nFilters*2,3)
    
    up4 = Conv2DTranspose(nFilters,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(block7)
    cat4 = concatenate([block1, up4])
    dr4 = Dropout(0.1)(cat4)
    block8 = get_conv_block(dr4,nFilters,3)

    conv10 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(block8)
    conv11 = Conv2D(3,(1,1), activation='softmax', padding = 'same',kernel_regularizer=l2(1e-4))(conv10)

    return (conv11, block7, block6, block5)

def get_model(input_shape,nFilters1,nFilters2):

    input_layer = Input(shape=input_shape)
    out1,out2,out3,out4 = get_UNET([input_layer],nFilters1,0)

    out1,out2,out3,out4 = get_UNET([out1,out2,out3,out4],nFilters2,1)

    model = Model(input_layer,out1)
    return model

In [ ]:
PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Ki67"


X1  = np.load(PATH + '/Ki67 IHC 230 Images.npy')
Y1  = np.load(PATH + '/Ki67 IHC 230 Masks.npy')

X2  = np.load(PATH + '/Ki67 IHC 232 Images.npy')
Y2  = np.load(PATH + '/Ki67 IHC 232 Masks.npy')

X3  = np.load(PATH + '/Ki67 IHC 242 Images.npy')
Y3  = np.load(PATH + '/Ki67 IHC 242 Masks.npy')

X4  = np.load(PATH + '/Ki67 IHC 263 Images.npy')
Y4  = np.load(PATH + '/Ki67 IHC 263 Masks.npy')

X5 = np.load(PATH + '/Ki67 IHC 221 Images.npy')
Y5 = np.load(PATH + '/Ki67 IHC 221 Masks.npy')

X6 = np.load(PATH + '/Ki67 IHC 229 Images.npy')
Y6 = np.load(PATH + '/Ki67 IHC 229 Masks.npy')

X7 = np.load(PATH + '/Ki67 IHC 239 Images.npy')
Y7 = np.load(PATH + '/Ki67 IHC 239 Masks.npy')

X8 = np.load(PATH + '/Ki67 IHC 246 Images.npy')
Y8 = np.load(PATH + '/Ki67 IHC 246 Masks.npy')

X9 = np.load(PATH + '/Ki67 IHC 252 Images.npy')
Y9 = np.load(PATH + '/Ki67 IHC 252 Masks.npy')

In [ ]:
print(X1.shape[0]/48)
print(X2.shape[0]/48)
print(X3.shape[0]/48)
print(X4.shape[0]/48)
print(X5.shape[0]/48)
print(X6.shape[0]/48)
print(X7.shape[0]/48)
print(X8.shape[0]/48)
print(X9.shape[0]/48)

10.0
10.0
11.0
11.0
10.0
10.0
10.0
10.0
10.0


In [ ]:
X = [X1, X2, X3, X4, X5, X6, X7, X8, X9]
Y = [Y1, Y2, Y3, Y4, Y5, Y6, Y7, Y8, Y9]
patient_no = ['230','232','242','263','221','229','239','246','252']
size = [10,10,11,11,10,10,10,10,10]

In [ ]:
fold_split = []

#test patient = 230, 232 
fold_split.append(([2,3,4,5,6,7,8],[0,1]))

#test_patient = 239, 242
fold_split.append(([0,1,3,4,5,7,8],[2,6]))

#test_patient = 252,263
fold_split.append(([0,1,2,4,5,6,7],[3,8]))

In [ ]:
def convertToLabels(data):
  data[data==128]=1
  data[data==255]=2

def convertFromLabels(data):
  data[data==1]=128
  data[data==2]=255

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

In [ ]:
def tversky_loss_1(y_true, y_pred):
    alpha = 0.7
    beta  = 0.3
    
    ones = K.ones(K.shape(y_true))
    p0 = y_pred      # proba that voxels are class i
    p1 = ones-y_pred # proba that voxels are not class i
  
    g0 = y_true
    g1 = ones-y_true
    
    num = K.sum(p0*g0, (0,1,2))
    den = num + alpha*K.sum(p0*g1,(0,1,2)) + beta*K.sum(p1*g0,(0,1,2))
    # 
    T = K.sum(num/den) # when summing over classes, T has dynamic range [0 Ncl]
    
    Ncl = K.cast(K.shape(y_true)[-1], 'float32')
    return Ncl-T

def tversky_loss_2(y_true, y_pred):
    alpha = 0.3
    beta  = 0.7
    
    ones = K.ones(K.shape(y_true))
    p0 = y_pred      # proba that voxels are class i
    p1 = ones-y_pred # proba that voxels are not class i
    g0 = y_true
    g1 = ones-y_true
    
    num = K.sum(p0*g0, (0,1,2))
    den = num + alpha*K.sum(p0*g1,(0,1,2)) + beta*K.sum(p1*g0,(0,1,2))
    # 
    T = K.sum(num/den) # when summing over classes, T has dynamic range [0 Ncl]
    
    Ncl = K.cast(K.shape(y_true)[-1], 'float32')
    return Ncl-T

def focal_tversky_loss_1(y_true, y_pred, gamma=0.75):
    tversky_index = tversky_loss_1(y_true, y_pred)
    return K.pow((tversky_index), gamma)
  
def focal_tversky_loss_2(y_true, y_pred, gamma=0.75):
    tversky_index = tversky_loss_2(y_true, y_pred)
    return K.pow((tversky_index), gamma)

def combined_loss(y_true, y_pred):
  return (0.2*K.categorical_crossentropy(y_true, y_pred))+(0.8*focal_tversky_loss_1(y_true, y_pred)+(0.8*focal_tversky_loss_2(y_true, y_pred)))

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

In [ ]:
optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0, amsgrad=True)

In [ ]:
batch_size = 8
def get_batch(batch_size, X_train, Y_train): 
    size_batch = batch_size
    last_index = len(X_train) - 1
    x_train = X_train
    y_train = Y_train 
    while True:
        batch_data = [[],[]]
        for i in range(0, size_batch):
            random_index = random.randint(0, last_index)
            batch_data[0].append(x_train[random_index])
            batch_data[1].append(y_train[random_index])

        yield (np.array(batch_data[0]), np.array(batch_data[1]))     

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
MODELS_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Models/Ki67/Combined FTL"

fold = 2

for train_index, test_index in fold_split:
  print("TRAIN:", train_index, "TEST:", test_index)
  if(test_index[0]==0 or test_index[0]==2):
    continue
  
  TrainX =  np.concatenate(np.array([X[i] for i in train_index]))
  TrainY =  np.concatenate(np.array([Y[i] for i in train_index]))
  TestX  =  np.concatenate(np.array([X[i] for i in test_index]))
  TestY  =  np.concatenate(np.array([Y[i] for i in test_index]))

  # Train = list(zip(TrainX,TrainY))
  # random.shuffle(Train)
  # TrainX,TrainY = zip(*Train)
  # TrainX = np.asarray(list(TrainX))
  # TrainY = np.asarray(list(TrainY))

  # Test = list(zip(TestX,TestY))
  # random.shuffle(Test)
  # TestX,TestY = zip(*Test)
  # TestX = np.asarray(list(TestX))
  # TestY = np.asarray(list(TestY))

  TrainX = TrainX.astype('float32')/255
  TestX = TestX.astype('float32')/255
  convertToLabels(TrainY)
  TrainY = keras.utils.to_categorical(TrainY,num_classes=3,dtype='int16')
  convertToLabels(TestY)
  TestY = keras.utils.to_categorical(TestY,num_classes=3,dtype='int16')
  ValX = TrainX[(int)(0.8*TrainX.shape[0]):]
  ValY = TrainY[(int)(0.8*TrainY.shape[0]):]
  model = get_model((240,240,3),32,64)

  
  mc = ModelCheckpoint(MODELS_PATH +'/Checkpoints/Checkpoint '+str(fold) +'.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0, amsgrad=True)

  model.compile(loss=combined_loss, optimizer= optimizer , metrics=[dice_coef,'accuracy']) 
  num_epoch = 100
  datagen = get_batch(batch_size, TrainX, TrainY)
  n_points = len(TrainX)
  print('-----------fold {}--------------'.format(fold))
  history = model.fit(datagen, validation_data = [ValX, ValY],
                  epochs=num_epoch,steps_per_epoch = math.ceil(n_points / batch_size), callbacks =[mc],  shuffle =True)
  model.save(MODELS_PATH + '/Ki67 '+ str(fold) +'.h5')
  fold = fold + 1

In [ ]:
def stitchMaskPatches(pieces):
  k = 0
  reconstructed_img = np.ones([1440,1920])
  for r in range(6):
    row = r * 240
    for c in range(8):
      col = c * 240
      reconstructed_img[row:row+240,col:col+240] = pieces[k]
      k = k + 1
  return reconstructed_img


def stitchImagePatches(pieces):
  k = 0
  reconstructed_img = np.ones([1440,1920,3])
  for r in range(6):
    row = r * 240
    for c in range(8):
      col = c * 240
      reconstructed_img[row:row+240,col:col+240,:] = pieces[k]
      k = k + 1
  return reconstructed_img

In [ ]:
def saveNumpyOutput(mask, Patient_array,Patient_length,title,folder):
  idx = 0
  for i in range(len(Patient_length)):
    temp = []
    for j in range(Patient_length[i]):
      final_output = mask[idx:idx+48]
      idx = idx + 48
      final_output = stitchMaskPatches(final_output)
      temp.append(final_output)
    final_output = np.asarray(temp)
    np.save(folder + title + Patient_array[i], final_output)

In [ ]:
MODELS_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Models/Ki67/Combined FTL/Ki67/"
model_names = os.listdir(MODELS_PATH)
print(model_names)


['Ki67 0.h5', 'Ki67 2.h5', 'Ki67 1.h5']


In [ ]:
SAVE_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Ki67/3Fold LadderNet/"

In [ ]:
# Axis 0 = Folds
# Axis 1 = Class (Backround, Immunopositive,Immunongative)
# Axis 2 = Evaluation Technique (Precision, Recall, Dice coefficient)
pixEvaluationTrain = np.empty((0,3,3))
pixEvaluationTest = np.empty((0,3,3))
score = np.zeros((1,3,3))
print(pixEvaluationTrain.shape)
print(pixEvaluationTest.shape)
print(score.shape)

(0, 3, 3)
(0, 3, 3)
(1, 3, 3)


In [ ]:

fold = 0

for train_index, test_index in fold_split:
 
  print("FOLD:",fold,"\tTRAIN:", train_index, "TEST:", test_index)
 #Splitting Train and Test data
  TrainX =  np.concatenate(np.array([X[i] for i in train_index]))
  TrainY =  np.concatenate(np.array([Y[i] for i in train_index]))
  TestX  =  np.concatenate(np.array([X[i] for i in test_index]))
  TestY  =  np.concatenate(np.array([Y[i] for i in test_index]))

  train_no = [patient_no[i] for i in train_index]
  train_size = [size[i] for i in train_index]
  print(train_no)

  test_no = [patient_no[i] for i in test_index]
  test_size = [size[i] for i in test_index]
  print(test_no)

  TrainX = TrainX.astype('float32')/255
  TestX = TestX.astype('float32')/255

  convertToLabels(TrainY)
  convertToLabels(TestY)

  TrainY = keras.utils.to_categorical(TrainY,num_classes=3, dtype='int16')
  TestY  = keras.utils.to_categorical(TestY, num_classes=3, dtype='int16')
  
  # Loading corrosponding fold of model
  model = keras.models.load_model(MODELS_PATH + model_names[fold],custom_objects={ 'combined_loss': combined_loss, 'dice_coef': dice_coef })

  # Predicting results using model
  trainResult = model.predict(TrainX, batch_size=8)
  testResult = model.predict(TestX,batch_size=8)

  trainResult = np.argmax(trainResult,axis=-1)
  testResult = np.argmax(testResult,axis=-1)

  trainOneHot = keras.utils.to_categorical(trainResult,num_classes=3, dtype='int16')
  testOneHot = keras.utils.to_categorical(testResult,  num_classes=3, dtype='int16')

  # Obtaining pixel-wise results: Precision recall and dice coefficient
  # For TRAIN
  y_true = np.reshape(TrainY,(-1, TrainY.shape[3]))
  y_pred = np.reshape(trainOneHot,(-1, trainOneHot.shape[3]))

  prec   = metrics.precision_score(y_true,y_pred,labels=[0,1,2],average=None,zero_division=1)
  recall = metrics.recall_score(y_true,y_pred,labels=[0,1,2],average=None,zero_division=1)
  dice = 2*prec*recall/(prec+recall)

  
  #Precision Recall Dice for each class  #0 : Background, 1 : Immunonegative, 2:Immunopositive 
  for i in range(3):
    score[0][i][0] = prec[i]
    score[0][i][1] = recall[i]
    score[0][i][2] = dice[i]

  pixEvaluationTrain = np.append(pixEvaluationTrain,score,axis=0)
  print(pixEvaluationTrain)
 
  # For TEST
  y_true = np.reshape(TestY,(-1, TestY.shape[3]))
  y_pred = np.reshape(testOneHot,(-1, testOneHot.shape[3]))

  prec   = metrics.precision_score(y_true,y_pred,labels=[0,1,2],average=None,zero_division=1)
  recall = metrics.recall_score(y_true,y_pred,labels=[0,1,2],average=None,zero_division=1)
  dice = 2*prec*recall/(prec+recall)

  # Precision Recall Dice for each class
  for i in range(3):
    score[0][i][0] = prec[i]
    score[0][i][1] = recall[i]
    score[0][i][2] = dice[i]

  pixEvaluationTest = np.append(pixEvaluationTest,score,axis=0)
  print(pixEvaluationTest)

#   # Converting predicted labels to required output format
#   convertFromLabels(trainResult)
#   convertFromLabels(testResult)

#  # Saving numpy arrays
#   path = os.path.join(SAVE_PATH,"Fold " + str(fold))
#   os.mkdir(path)
#   os.mkdir(path + '/Train')
#   os.mkdir(path + '/Test')

#   saveNumpyOutput(trainResult, train_no, train_size,"/Train/",path)
#   saveNumpyOutput(testResult, test_no, test_size,"/Test/",path)

  fold +=1

FOLD: 0 	TRAIN: [2, 3, 4, 5, 6, 7, 8] TEST: [0, 1]
['242', '263', '221', '229', '239', '246', '252']
['230', '232']
[[[0.95641806 0.96895468 0.96264556]
  [0.87621469 0.84015417 0.85780562]
  [0.90193241 0.85918155 0.88003809]]]
[[[0.92102706 0.94215552 0.93147149]
  [0.76701273 0.75982919 0.76340407]
  [0.89976114 0.72169478 0.80095044]]]
FOLD: 0 	TRAIN: [0, 1, 3, 4, 5, 7, 8] TEST: [2, 6]
['230', '232', '263', '221', '229', '246', '252']
['242', '239']
[[[0.95641806 0.96895468 0.96264556]
  [0.87621469 0.84015417 0.85780562]
  [0.90193241 0.85918155 0.88003809]]

 [[0.95019373 0.96422227 0.9571566 ]
  [0.83326969 0.80874431 0.82082384]
  [0.90573671 0.82040347 0.86096083]]]
[[[0.92102706 0.94215552 0.93147149]
  [0.76701273 0.75982919 0.76340407]
  [0.89976114 0.72169478 0.80095044]]

 [[0.94426542 0.96001749 0.9520763 ]
  [0.88811384 0.85019871 0.86874279]
  [0.89446372 0.85813998 0.87592544]]]
FOLD: 0 	TRAIN: [0, 1, 2, 4, 5, 6, 7] TEST: [3, 8]
['230', '232', '242', '221', '229', '23

In [ ]:
print(pixEvaluationTrain.shape)
print(pixEvaluationTest.shape)

(3, 3, 3)
(3, 3, 3)


In [ ]:
np.save(SAVE_PATH + "Pixel-wise Metrics Train.npy", pixEvaluationTrain)
np.save(SAVE_PATH + "Pixel-wise Metrics Test.npy", pixEvaluationTest)

In [ ]:
LOAD_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Ki67/KFold LadderNet/"
trainMetrics = np.load(LOAD_PATH + "Pixel-wise Metrics Train.npy")
testMetrics  = np.load(LOAD_PATH + "Pixel-wise Metrics Test.npy")
print(trainMetrics.shape)
print(testMetrics.shape)


(3, 3, 3)
(3, 3, 3)


In [ ]:

#For pixel-wise background is included and for object-wise background is not included
numClasses = 3
avgTrainMetrics  = np.zeros([numClasses, 3])
avgTestMetrics   = np.zeros([numClasses, 3])

def calculateMetrics(AvgMetrics, ValidationMetrics):
  for patientNum in range(ValidationMetrics.shape[0]):
    for row in range(numClasses):
      for col in range(3):
        AvgMetrics[row][col] += ValidationMetrics[patientNum][row][col]
  AvgMetrics = np.divide(AvgMetrics,ValidationMetrics.shape[0] )
  return AvgMetrics


avgTrainMetrics = calculateMetrics(avgTrainMetrics, trainMetrics)
print('TRAIN :')
print(avgTrainMetrics)

avgTestMetrics = calculateMetrics(avgTestMetrics, testMetrics)
print('TEST :')
print(avgTestMetrics)

TRAIN :
[[0.9558469  0.957932   0.95671614]
 [0.8550969  0.81991115 0.83443751]
 [0.86590669 0.89296803 0.87713484]]
TEST :
[[0.93093541 0.93941124 0.93411226]
 [0.79153604 0.66190771 0.68281195]
 [0.83019238 0.82337968 0.82415122]]
